In [1]:
"""
4.1.3 LangChain/LangGraph Orchestration
We will use LangChain and LangGraph (from Modules 4, 9, 10) to manage the workflow.
LangGraph will handle the conversation flow by checking question safety, retrieving
relevant passages, generating a response, validating response safety, and sending it to the
user. If something fails the safety checks, the system can try again or use a backup
response.
"""

'\n4.1.3 LangChain/LangGraph Orchestration\nWe will use LangChain and LangGraph (from Modules 4, 9, 10) to manage the workflow.\nLangGraph will handle the conversation flow by checking question safety, retrieving\nrelevant passages, generating a response, validating response safety, and sending it to the\nuser. If something fails the safety checks, the system can try again or use a backup\nresponse.\n'

In [2]:
!pip3 install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.


In [4]:
# import statements
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition

import re, math
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import time
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import openai
import os

from transformers import pipeline
from typing import Dict, List, Set, Tuple

In [5]:
from input_filter import InputFilter, filter_input
from output_filter import OutputFilter, filter_output

In [6]:
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")

### Ada RAG and GPT:

In [7]:
# abot how many words per passage / chunk
passage_size = 120

def passage_text(text, passage_size=passage_size):
  words = text.split()
  passages = []
  for i in range(0, len(words), passage_size):
    passage = " ".join(words[i:i+passage_size]).strip()
    if len(passage) > 20:
      passages.append(passage)
  return passages

data = "cleaned_merged_fairy_tales_without_eos.txt"

# try to split into candidate stories
with open(data, "r", encoding="utf-8", errors="replace") as fh:
  raw = fh.read()

# split on multiple newlines
story_blocks = [b.strip() for b in re.split(r'\n{2,}', raw) if len(b.strip())>50]
print(f"Detected {len(story_blocks)} story-like blocks (heuristic)")

rows = []
for si, block in enumerate(story_blocks):
  title = f"story_{si}"
  passages = passage_text(block)
  for j, c in enumerate(passages):
    rows.append({"story_id": si, "title": title, "passage_id": f"{si}_{j}", "passage": c})

passages_df = pd.DataFrame(rows)
print("Total passages:", len(passages_df))
print()
output_dir = Path("./output_data")
output_dir.mkdir(parents=True, exist_ok=True)
passages_df.to_csv(output_dir/"passages.csv", index=False)

Detected 908 story-like blocks (heuristic)
Total passages: 32033



In [9]:
model_name = "all-MiniLM-L6-v2"
embed_model = SentenceTransformer(model_name)

# check gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
embed_model = embed_model.to(device)

passages = passages_df['passage'].tolist()
total_passages = len(passages)

# compute embeddings with batching
batch_size = 128
embeddings = []

start_time = time.time()
for i in range(0, total_passages, batch_size):
  batch = passages[i:i+batch_size]
  batch_embeddings = embed_model.encode(
      batch,
      convert_to_numpy=True,
      show_progress_bar=True
  )
  embeddings.append(batch_embeddings)

# combine all batches
embeddings = np.vstack(embeddings)
end_time = time.time()
print("Embedding computation time:", round(end_time - start_time, 1), "s")

# save final embeddings
np.save(output_dir / "embeddings.npy", embeddings)
passages_df.to_pickle(output_dir / "passages_meta.pkl")
print("Saved final embeddings and passages to:", output_dir)

Using device: cpu


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding computation time: 2117.4 s
Saved final embeddings and passages to: output_data


In [10]:
def retrieve_top_passages(query, embeddings, passages_df, top_k=3):
  query_vec = embed_model.encode([query])
  sims = cosine_similarity(query_vec, embeddings)[0]
  top_indices = sims.argsort()[-top_k:][::-1]
  return passages_df.iloc[top_indices]

In [100]:
client = openai.OpenAI()

# define gpt query function
# MODIFIED to take message history
def ask_gpt(query, retrieved_passages, message_history, model="gpt-4o-mini", max_tokens=300, temperature=0.7):
  if isinstance(retrieved_passages, pd.DataFrame) and not retrieved_passages.empty:
      context_text = "\n\n".join(retrieved_passages['passage'].tolist())
  else:
      context_text = "No relevant passages found."

  prompt = f"""
      You are a friendly reading assistant for children ages 6 to 10.
      Use the passages below to answer the child's question.
      Always ask 1–2 follow-up questions to engage critical thinking.
      """

  messages = [{"role":"system", "content": prompt}]

  for message in message_history[-5:]:
      messages.append(message)

  recent_query = f"""
  Passages:
  {context_text}

  Child's question: {query}

  Answer:
  """

  messages.append({"role":"user", "content":recent_query})

  # call gpt
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=temperature,
      max_tokens=max_tokens
  )

  # extract and return text
  return response.choices[0].message.content.strip()


### Creating Message State Tool Specified for Use Case

In [89]:
# in recitation, we saw an example with Messages State,
# but that does not suit the specific needs of our project
# so we define our own below:
from typing import TypedDict, Any

class ChildMessagesState(TypedDict):
    # the user input
    user_query: str
    # to store message history
    # TODO: only store history up to a certain amount?
    # then reset
    messages: list
    # the RAG results
    retrieve_passages: Any
    # system initial response
    response: str
    # system output after safety check
    final_output: str
    # counts how many questions have been asked
    # limit 10 to limit screen time
    turn_count: int

### Creating Nodes for Graph

In [90]:
# get user input

def get_user_input(state: ChildMessagesState):
  user_input = state['user_query']

  # add one to the turn count
  state['turn_count'] += 1

  # store in message history, under role of user since user input
  message_history = state['messages'] + [{"role": "user", "content": user_input}]

  return {"turn_count": state['turn_count'], "messages":message_history, "user_query":user_input}


In [91]:
# check question safety --> use function from Mia's safety filtering section

def check_input_safety(state: ChildMessagesState):
  user_input = state['user_query']

  # use safety filtering to check if input is safe
  # if it is not, fall back on sfae response
  if filter_input(user_input) == False:
    state['final_output'] = """I'm sorry, that response is not safe for me to answer.
    Please ask a trusted adult for further information on this question.
    What else would you like to learn about?"""

    # add system response to history as it is final response automatically
    messages_history = state['messages'] + [{"role": "system", "content": state['final_output']}]

    return {"final_output":state['final_output'],"messages":messages_history,"END":True}

  return {}


In [92]:
# make sure that user is not spending too long on system
# within certain time period? TODO

def check_exchange_count(state: ChildMessagesState):

  # if we have reached limit, return safe message as final output
  if state['turn_count'] >= 10:
    final_output = "It was great talking to you! However, your librAIrian \
    needs to take a break. Come back later to ask more questions!"

    # add to history
    message_history = state['messages'] + [{"role": "system", "content": state['final_output']}]

    return {"final_output":final_output, "messages":message_history, "END":True}

  return {}


In [93]:
# retrieve relevant passages

def retrieve_passages_node(state: ChildMessagesState):
  # using rag to retrieve passages
  results = retrieve_top_passages(state['user_query'], embeddings, passages_df)
  state['retrieve_passages'] = results

  return {'retrieve_passages':results}


In [95]:
# generate response

def generate_response(state: ChildMessagesState):
  # generate_answer from ada
  # call it using the query, the results of RAG, and message history
  response = ask_gpt(
      state['user_query'],
      state['retrieve_passages'],
      state['messages']
    )

  state['response'] = response
  state['messages'].append({"role": "system", "content": response})

  return {"response":response, "messages":state['messages']}


In [96]:
# validate response safety

# if answer is unsafe, only try again a set number of times
max_attempts = 3

def check_answer_safety(state: ChildMessagesState):
  attempts = 0
  while attempts < max_attempts:
    # output safety check from Mia's safety filtering layer
    if filter_output(state['response']) == False:
      print("""Please wait one second for me to rephrase my response!
      My initial response was not safe.""")

      # try generating a new answer
      # and then reenter this loop
      response = ask_gpt(
        state['user_query'],
        state['retrieve_passages'],
        state['messages']
      )
      attempts += 1

    # if the output is safe, then the draft response is the final response
    else:
      final_output = state['response']
      # add to message history
      message_history = state['messages'] + [{"role": "system", "content": state['final_output']}]
      return {"final_output":final_output, "messages":message_history}

  # if we exit the while loop because exceeded max attempts
  # give back up message
  final_output = "I'm sorry, I reached the maximum number of attempts to \n  generate a safe response. Please try another question."
  # add this to message history for system
  message_history = state['messages'] + [{"role": "system", "content": state['final_output']}]
  return {"final_output":final_output, "messages":message_history}


### Creating Actual Graph

In [103]:
# create a graph builder to set nodes and edges using our messages state
# modeled this after the example from lab 7

graph_builder = StateGraph(ChildMessagesState)

In [104]:
# add nodes, and give them the same name
graph_builder.add_node("get_user_input", get_user_input)
graph_builder.add_node("check_input_safety", check_input_safety)
graph_builder.add_node("check_exchange_count", check_exchange_count)
graph_builder.add_node("retrieve_passages_node", retrieve_passages_node)
graph_builder.add_node("generate_response", generate_response)
graph_builder.add_node("check_answer_safety", check_answer_safety)

# add edges between subsequent pieces of the pipeline
graph_builder.set_entry_point("get_user_input")
graph_builder.add_edge("get_user_input", "check_input_safety")
graph_builder.add_edge("check_input_safety", "check_exchange_count")
graph_builder.add_edge("check_exchange_count", "retrieve_passages_node")
graph_builder.add_edge("retrieve_passages_node", "generate_response")
graph_builder.add_edge("generate_response", "check_answer_safety")
graph_builder.add_edge("check_answer_safety", END)

In [106]:
# compile into a single graph object
graph = graph_builder.compile()

for i in range(10):
    # get user input
    user_query = str(input("How can LibrAIrian help you? "))

    # initialize state object
    state = ChildMessagesState(
        user_query=user_query,
        messages=[],
        retrieve_passages=None,
        response="",
        final_output="",
        turn_count=0
    )

    # set user query to input from above
    state['user_query'] = user_query
    # generate response by invoking the graph we built
    output = graph.invoke(state)
    # print the final result to the user
    print(output['final_output'])

How can LibrAIrian help you? why are princesses cool?
Loading toxic-BERT model for input filtering...


Device set to use cpu


Input filter ready!
Loading models for output filtering...
  - Loading toxic-BERT...


Device set to use cpu


  - Loading BART for topic detection...


...

Input filter ready!
Loading models for output filtering...
  - Loading toxic-BERT...


Device set to use cpu


  - Loading BART for topic detection...


Device set to use cpu


  - Loading Dale-Chall vocabulary from dale_chall_words.txt...
    Loaded 2940 words
Output filter ready
It sounds like the characters in the passage were having a lot of fun and getting into some funny situations! Acting silly can lead to laughter and unexpected adventures. For example, when they fell over the turkey-pen and ended up in a ridiculous scene with the turkeys, it shows how their silliness created a funny moment.

What do you think makes something silly? Can you think of a time when you or your friends acted silly and it made you laugh?
